In [120]:
import pandas as pd

# Pandas Test - Real life exercise

For visualizations:

In [121]:
import cufflinks as cf; cf.go_offline()
import plotly_express as px

## About the data

The data you are about to load is coming from an Ad recommentation system of the company InBrain. InBrain is a adTech company and specalize in next-level ad recommendation.  
The company has two major products - one giving high quality recommendation while the other is giving an highly yielding recommendation. Inbrain custormers are sending requests, asking for either of the products. 

Once a week, the company is doing an internal quality assements, and sends a sample of its traffic requests to external annotation process, in which the annotators are trained to manually decided whether the recommended Ad is a succes or not. The data contains information about the ad (**ad_id**), the sampled week (**week_id**), the requester company (**requester**), the region of operation (**region**), the recommendation type (Quality/Yield, **recommendation_type**) and the recommendation provider algorithm (**rec_provider**). The annotators are marking whether the recommendation is meeting the Quality/Yield standards (**is_success**) and whether or not the recommendation is severly defected (**is_sever**)

See a sample below:

In [122]:
ad_recs_annotated = pd.read_csv('data/ad_annotations.csv')
ad_recs_annotated.sample(9)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider
228909,2021-week_48,AD078HGTZX,Extreme DA,US,yes,NaN,Yield,DNNY
103293,2021-week_40,AD008RYNN8,VarVee,GB,yes,NaN,Yield,DNNY
51647,2021-week_36,AD07ZJND99,Puentes Company,IN,no,False,Yield,UserPopY
172080,2021-week_44,AD00K8O8IK,Glory Medical,US,no,True,Yield,RuleBasedY
91954,2021-week_39,AD07R8PL8R,Puentes Company,IN,yes,NaN,Yield,UserPopY
242422,2021-week_49,AD00M90HTM,Fancy,US,yes,NaN,Qality,DNNQ
16757,2021-week_34,AD00SP88KZ,Bizanga,GB,yes,NaN,Yield,DNNY
179672,2021-week_45,AD078C8HY,RelayFoods,IN,yes,NaN,Qality,UserPopQ
371034,2022-week_52,AD08DTZQPT,Extreme DA,US,yes,NaN,Yield,DNNY


Your job, as the new and only data scientist of the company, is to get familiar with the the data, show week-over-week trends and produce insightfull graphs as a preparation to a full blown BI dashboard.

## Questions

### Data Modifications

* Add a column with the sample year
* Add a column with the sample quarter (google how to)
* Add a new success column with values 1 and 0 for yes and no 
* Are there duplicated ads? To compac the data, remove duplicated ads and instead add an ad_count column **(pay attention, this is not trivial at all)**
* Are there any NaNs in the is_sever column? Count how many and create a new column with NaNs filled as False. Check that indeed the new column contaion no NaNs. 
* Capitlize (first letter only) the is_success column

In [123]:
ad_recs_annotated['week_id'] = ad_recs_annotated['week_id'].replace('2022-week_52','2021-week_52') #Just fixing the dataset, not an answer

In [124]:
ad_recs_annotated['week'] = ad_recs_annotated['week_id'].astype(str).str[-2 :].astype(int)


In [125]:
ad_recs_annotated['sample_year'] = pd.DatetimeIndex(pd.to_datetime(ad_recs_annotated['week_id'].astype(str).str[0 : 4],format='%Y')).year
ad_recs_annotated['sample_year'] 

0         2021
1         2021
2         2021
3         2021
4         2021
          ... 
373783    2021
373784    2021
373785    2021
373786    2021
373787    2021
Name: sample_year, Length: 373788, dtype: int64

In [126]:
ad_recs_annotated['quarter'] = ad_recs_annotated['week'].astype(str).str[-2 :].astype(int)
ad_recs_annotated.loc[(ad_recs_annotated['week'] < 13) , 'quarter'] = 'Q1'
ad_recs_annotated.loc[((ad_recs_annotated['week'] >= 13) & (ad_recs_annotated['week'] < 26)) , 'quarter'] = 'Q2'
ad_recs_annotated.loc[((ad_recs_annotated['week'] >= 26) & (ad_recs_annotated['week'] < 38)) , 'quarter'] = 'Q3'
ad_recs_annotated.loc[((ad_recs_annotated['week'] >= 38) & (ad_recs_annotated['week'] < 53)) , 'quarter'] = 'Q4'

In [127]:
ad_recs_annotated.sample(5)

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,week,sample_year,quarter
347026,2022-week_06,AD07PJF7HK,Fancy,GB,no,True,Qality,DNNQ,6,2022,Q1
65538,2021-week_37,AD07ZF8QXJ,Puentes Company,IN,yes,NaN,Yield,UserPopY,37,2021,Q3
151527,2021-week_43,AD098P8MK7,Puentes Company,IN,yes,NaN,Yield,UserPopY,43,2021,Q4
203338,2021-week_47,AD0088X868,Search Million Culture,DE,no,True,Qality,DNNQ,47,2021,Q4
54109,2021-week_36,AD00MJ7VL8,Fry Multimedia,US,yes,NaN,Qality,ManualQ,36,2021,Q3


In [128]:
ad_recs_annotated['is_success_binary'] = ad_recs_annotated['is_success'].replace('yes',1).replace('no',0)
ad_recs_annotated['is_success_binary'].sample(10)

41813     1
84143     1
109371    0
340517    0
64245     1
331123    1
219405    1
95727     1
220923    1
188901    1
Name: is_success_binary, dtype: int64

In [129]:
sum(ad_recs_annotated['is_sever'].isna())

304195

In [130]:
ad_recs_annotated['is_severe_corrected'] = ad_recs_annotated['is_sever'].fillna(False)
ad_recs_annotated['is_severe_corrected'].sample(5)

355424     True
42875     False
86073     False
237853    False
370909    False
Name: is_severe_corrected, dtype: bool

In [131]:
ad_recs_annotated['ad_id'].value_counts()

AD07PFFMP9    247
AD098SWYF6    239
AD08C8RR8J    138
AD0886VY87     99
AD08888888     96
             ... 
AD07Y8JSHJ      1
AD0888J7AO      1
AD087G7SHR      1
AD086QCZGJ      1
AD08FWNFDO      1
Name: ad_id, Length: 300727, dtype: int64

In [132]:
ad_recs_annotated['is_success'].str.capitalize()

0         Yes
1         Yes
2         Yes
3         Yes
4         Yes
         ... 
373783    Yes
373784    Yes
373785    Yes
373786    Yes
373787    Yes
Name: is_success, Length: 373788, dtype: object

### Subset selection

for each question, sub-select the data by using the conditional selection (`[]`) and the `.query` API. Use `.shape` on the subselection to obtain the result.  
For example:
`df.query('some_condition').shape`

1. How many requests are there in US region? 
1. How many **successful** requests are there in US region? 
1. How many **successful** requests are there in US region, on week 52?
1. How many requests where delivered by DDNQ, RRNY and UserPopQ together? 
1. How many requests where delivered by rule based providers? 
1. Select only the requests from 2021 - How many are there? 
1. Select the reqeusts from week 30 to week 40 - How many are there? 
1. Select all the data that comes from the most popular Ad
1. Select all the data that comes from the least popular provider
1. Select the data in which is_sever is None. How big is it? 
1. Select the data in which the requester is a 3 word company
1. Select the data in which the requester is a 3 word company, and the ad_id has the letter 6 in it
1. Select the data in which the requester is a 3 word company, and the multiplication of all the numbers in the ad_id is bigger than 30

In [133]:
#Q1
ad_recs_annotated.query('region == "US"').shape

(103846, 13)

In [134]:
#Q2
ad_recs_annotated.query('region == "US" and is_success_binary == 1').shape

(88918, 13)

In [135]:
#Q3
ad_recs_annotated.query('region == "US" and is_success_binary == 1')['week'].astype(str).str[-2 :].astype(int)

10382     33
10383     33
10384     33
10385     33
10386     33
          ..
373783    52
373784    52
373785    52
373786    52
373787    52
Name: week, Length: 88918, dtype: int32

In [136]:
#Q4
ad_recs_annotated.query('rec_provider == "RuleBasedY"').shape

(28154, 13)

In [137]:
#Q5
len(ad_recs_annotated.query('rec_provider == ["DDNQ","RRNY","UserPopQ"]'))

69937

In [138]:
#Q6
ad_recs_annotated.query('sample_year == 2021').shape

(287545, 13)

In [139]:
#Q7
sum(ad_recs_annotated['week_id'].astype(str).str[-2:].astype(int).between(30,40))

115051

In [140]:
#Q8
ad_recs_annotated.groupby('is_success_binary').max()

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,week,sample_year,quarter,is_severe_corrected
is_success_binary,,,,,,,,,,,,
0,2022-week_06,AD99798888,iDreamsky Technology,US,no,True,Yield,XGBY,52,2022,Q4,True
1,2022-week_06,ADOO8X898E,iDreamsky Technology,US,yes,True,Yield,XGBY,52,2022,Q4,True


In [141]:
#Q9
ad_recs_annotated[['requester','is_success_binary']].query('is_success_binary == 0').groupby('is_success_binary')['requester'].value_counts().head(1)

is_success_binary  requester             
0                  Search Million Culture    16209
Name: requester, dtype: int64

In [142]:
#Q10
ad_recs_annotated.query('is_sever == False')

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,week,sample_year,quarter,is_success_binary,is_severe_corrected
44,2021-week_33,AD07FK8888,Search Million Culture,DE,no,False,Qality,BooksQ,33,2021,Q3,0,False
46,2021-week_33,AD00JG8D7U,SOLOMO365,DE,no,False,Qality,UserPopQ,33,2021,Q3,0,False
51,2021-week_33,AD80688889,Search Million Culture,DE,no,False,Qality,BooksQ,33,2021,Q3,0,False
66,2021-week_33,AD0988K8R9,Search Million Culture,DE,no,False,Qality,DNNQ,33,2021,Q3,0,False
68,2021-week_33,AD088P6887,Search Million Culture,DE,no,False,Qality,UserPopQ,33,2021,Q3,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
373704,2021-week_52,AD08L788VC,Extreme DA,US,no,False,Yield,RuleBasedY,52,2021,Q4,0,False
373729,2021-week_52,AD09MCYZCG,Extreme DA,US,no,False,Yield,RuleBasedY,52,2021,Q4,0,False
373730,2021-week_52,AD07JGDT8K,Extreme DA,US,no,False,Yield,RuleBasedY,52,2021,Q4,0,False
373738,2021-week_52,AD008N8MW8,Fancy,US,no,False,Qality,DNNQ,52,2021,Q4,0,False


In [143]:
#Q11
ad_recs_annotated['requester'][ad_recs_annotated['requester'].apply(str.split).apply(lambda x : len(x) == 3)]

0         Search Million Culture
1          MoJoe Brewing Company
2         Search Million Culture
3         Search Million Culture
4         Search Million Culture
                   ...          
373663     MoJoe Brewing Company
373689     MoJoe Brewing Company
373691     MoJoe Brewing Company
373722    Search Million Culture
373728     MoJoe Brewing Company
Name: requester, Length: 118141, dtype: object

In [144]:
#Q12
ad_recs_annotated['requester'][ad_recs_annotated['requester'].apply(str.split).apply(lambda x : len(x) == 3)][ad_recs_annotated['ad_id'].apply(lambda x : '6' in x)]

2         Search Million Culture
3         Search Million Culture
4         Search Million Culture
11        Search Million Culture
41         MoJoe Brewing Company
                   ...          
373364     MoJoe Brewing Company
373394     MoJoe Brewing Company
373511     MoJoe Brewing Company
373691     MoJoe Brewing Company
373722    Search Million Culture
Name: requester, Length: 24197, dtype: object

In [145]:
ad_recs_annotated['requester'][ad_recs_annotated['requester'].apply(str.split).apply(lambda x : len(x) == 3)][ad_recs_annotated['ad_id'].apply(lambda x : x.replace('[a-zA-Z]',''))apply(lambda x : list(float(x))).apply(lambda x, y : 1*y for y in x)]

SyntaxError: invalid syntax (1796885587.py, line 1)

### Analysis

1. How many weeks available in the data? Are there any missing weeks? 
1. How many regions available in the data? Are there any missing weeks per region? 
1. How many ads are being sent to annotation in each region per week?
1. How many None values are there in is_sever column? 
1. Are ads being sent more than one time in a given week? what is the ad_id that was sent the most in a given week? (e.g. ad X where sent 50 times in week Y)
1. Are there ads that are being sent in more than one week?
   1. Which is the ad that was sent in most amount of weeks (e.g. ad X where sent in Y different weeks)
   2. What are the weeks that the ad from above was sent in? 
   3. Is there an Ad that was succsefull in one week, but not succesfful in a different week? Show one.
1. When is_sever is None, what is the number of successful requests? What is the number of non-successful requests? What do you learn from it about the reason for Nones in the first place? 
1. Per each region, What is the Quality/Yield traffic proportion WoW? 
1. How many different reqeusters are there? 
1. Which are the top 5 requesters per region? 
1. Which are the different rec_proivders?
1. Are there different rec providers per region? 
1. Are there different rec providers per rec type?
1. What are the notation rules distinguishing between quality vs yielding providers?
1. Which is the most successful region of operation? 
1. Which is the most successful rec provider per each region? 
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec provider
1. Present a table with a success rate, sever defects rate and CI (for each metric) per region and rec provider combinations
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type (Q/Y)
1. Present a table with a success rate, sever defects rate and CI (for each metric) per rec type and region
1. Present WoW table/graph with success rate and CI (see image below) per region
1. Present WoW table/graph with success rate and CI (see image below) per region and rec type
1. Present WoW table/graph with success rate and CI (see image below) per region and rec provider
1. Which are the requester to which we succeed the most?
1. Which are the requester to which we produce the most critical defects?
1. What is the overall success rate trend over time? 
1. What is the overall sever defect rate trend over time? 
1. Preset a WoW graph showing the number of requests per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients
1. Preset a WoW graph showing the **requests proportion** per customer in each region (hint: [use stacked bars](https://plotly.com/python/bar-charts/)), from it:
  1. Identify major traffic shape changes (addition/removal) of major clients


Analysis Bonus questions:
1. Compute the per region success rate and CI in trailing weeks, Say, 4 weeks instead of 1 week - to allow for smoother estimations and smaller CIs

In [ ]:
#Q1
ad_recs_annotated['week_id'].nunique()

26

In [ ]:
#Q2
ad_recs_annotated['region'].nunique()

5

In [ ]:
#Q3
ad_recs_annotated.groupby(['region','week_id'])['region'].value_counts()

region  week_id       region
DE      2021-week_33  DE        2395
        2021-week_34  DE        2360
        2021-week_35  DE        2393
        2021-week_36  DE        2401
        2021-week_37  DE        2404
                                ... 
US      2022-week_02  US        4016
        2022-week_03  US        3979
        2022-week_04  US        3999
        2022-week_05  US        3998
        2022-week_06  US        3992
Name: region, Length: 129, dtype: int64

In [ ]:
#Q4
sum(ad_recs_annotated['is_sever'].isna())

304195

In [ ]:
#Q5
len(ad_recs_annotated.query('region == "US" and is_success_binary == 1'))

88918

In [ ]:
#Q6A
ad_recs_annotated.groupby(['week_id','ad_id'])['ad_id'].value_counts().sort_values(ascending=False).head(1)

week_id       ad_id       ad_id     
2021-week_40  AD098SWYF6  AD098SWYF6    81
Name: ad_id, dtype: int64

In [ ]:
#Q6B
ad_recs_annotated.query('ad_id == "AD098SWYF6"')

,week_id,ad_id,requester,region,is_success,is_sever,recommendation_type,rec_provider,week,sample_year,quarter,win,is_success_binary,is_severe_corrected
7261,2021-week_33,AD098SWYF6,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,33,2021,33,Q3,1,False
7262,2021-week_33,AD098SWYF6,Puentes Company,IN,yes,NaN,Yield,UserPopY,33,2021,33,Q3,1,False
8082,2021-week_33,AD098SWYF6,Search Million Culture,IN,yes,NaN,Qality,UserPopQ,33,2021,33,Q3,1,False
8083,2021-week_33,AD098SWYF6,Search Million Culture,IN,yes,NaN,Qality,UserPopQ,33,2021,33,Q3,1,False
20757,2021-week_34,AD098SWYF6,Search Million Culture,IN,yes,NaN,Qality,UserPopQ,34,2021,34,Q3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363581,2021-week_52,AD098SWYF6,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,52,2021,52,NaN,1,False
363582,2021-week_52,AD098SWYF6,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,52,2021,52,NaN,1,False
363583,2021-week_52,AD098SWYF6,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,52,2021,52,NaN,1,False
363584,2021-week_52,AD098SWYF6,MoJoe Brewing Company,IN,yes,NaN,Yield,UserPopY,52,2021,52,NaN,1,False


In [ ]:
#Q7
ad_recs_annotated[['requester','is_sever','is_success_binary']].query("is_sever == False").groupby('is_success_binary')['requester'].value_counts()

is_success_binary  requester                   
0                  Search Million Culture          6833
                   RelayFoods                      5255
                   Extreme DA                      4146
                   Puentes Company                 2705
                   MoJoe Brewing Company           2144
                   Fancy                            719
                   LocalVox Media                   579
                   iDreamsky Technology             575
                   Fry Multimedia                   379
                   Tab Solutions                    342
                   SOLOMO365                        294
                   Sensor Tower                     164
                   Modanisa                         158
                   Bizanga                          131
                   Crescendo Networks               111
                   VarVee                            99
                   G-mode                            96


In [ ]:
# Q8
Yield = ad_recs_annotated[['week_id','region','recommendation_type']].query("recommendation_type == 'Yield'").replace('Yield','Quality/Yield').groupby(['week_id','region'])['recommendation_type'].value_counts()
Quality = ad_recs_annotated[['week_id','region','recommendation_type']].query("recommendation_type == 'Qality'").replace('Qality','Quality/Yield').groupby(['week_id','region'])['recommendation_type'].value_counts()
Quality.divide(Yield)

week_id       region  recommendation_type
2021-week_33  DE      Quality/Yield           3.917864
              GB      Quality/Yield           5.360000
              IN      Quality/Yield           0.845266
              JP      Quality/Yield           4.613240
              US      Quality/Yield           1.948071
                                               ...    
2022-week_06  DE      Quality/Yield           5.830508
              GB      Quality/Yield           9.691275
              IN      Quality/Yield           1.350774
              JP      Quality/Yield          10.288732
              US      Quality/Yield           0.582243
Name: recommendation_type, Length: 129, dtype: float64

In [ ]:
#Q9
ad_recs_annotated['requester'].nunique()

36

In [ ]:
#Q10
ad_recs_annotated.groupby(['requester','region'])['region'].value_counts().head(5)

requester             region  region
Allthetopbananas.com  DE      DE         51
                      GB      GB         50
                      IN      IN        107
                      US      US          8
Altammune             US      US          5
Name: region, dtype: int64

In [ ]:
#Q11
ad_recs_annotated.groupby('rec_provider')['rec_provider'].nunique()

rec_provider
BooksQ               1
BooksY               1
DNNQ                 1
DNNY                 1
ManualQ              1
ManualY              1
RNNQ                 1
RNNY                 1
RuleBased            1
RuleBasedY           1
UserPopQ             1
UserPopSelectionQ    1
UserPopSelectionY    1
UserPopY             1
XGBQ                 1
XGBY                 1
Name: rec_provider, dtype: int64

In [ ]:
# Q12
ad_recs_annotated[['region','rec_provider']].groupby(['region']).nunique()

,rec_provider
region,
DE,9
GB,12
IN,6
JP,9
US,14


In [ ]:
# Q13
ad_recs_annotated[['rec_provider','recommendation_type']].groupby(['rec_provider']).nunique()

,recommendation_type
rec_provider,
BooksQ,1
BooksY,1
DNNQ,1
DNNY,1
ManualQ,2
ManualY,2
RNNQ,1
RNNY,1
RuleBased,1


In [ ]:
# Q15
ad_recs_annotated[['region','is_success_binary']].groupby(['is_success_binary']).max()

,region
is_success_binary,
0,US
1,US


In [ ]:
# Q16
ad_recs_annotated[['region','rec_provider','is_success_binary']].query("is_success_binary == 1").value_counts().head(5)

region  rec_provider  is_success_binary
GB      DNNQ          1                    34681
IN      UserPopQ      1                    30407
DE      DNNQ          1                    27215
US      DNNY          1                    25705
JP      DNNQ          1                    20113
dtype: int64

In [146]:
#Q21
issuccessratio = ad_recs_annotated[['region','recommendation_type','is_success_binary']].groupby(['region','recommendation_type'])['is_success_binary'].sum() / ad_recs_annotated[['region','recommendation_type','is_success_binary']].groupby(['region','recommendation_type'])['is_success_binary'].size()
CImin = issuccessratio.mean() - ((1.96*issuccessratio.std())/len(issuccessratio))
CImax = issuccessratio.mean() + ((1.96*issuccessratio.std())/len(issuccessratio))
print(f'Range of confidence index for is_success: {CImin} < CI < {CImax}')

Range of confidence index for is_success: 0.7865494796103423 < CI < 0.8115507249658964


In [147]:
#Q21
issevereratio = ad_recs_annotated[['region','recommendation_type','is_severe_corrected']].groupby(['region','recommendation_type'])['is_severe_corrected'].sum() / ad_recs_annotated[['region','recommendation_type','is_severe_corrected']].groupby(['region','recommendation_type'])['is_severe_corrected'].size()
CImin = issevereratio.mean() - ((1.96*issevereratio.std())/len(issevereratio))
CImax = issevereratio.mean() + ((1.96*issevereratio.std())/len(issevereratio))
print(f'Range of confidence index for is_severe: {CImin} < CI < {CImax}')

Range of confidence index for is_severe: 0.1167331737586564 < CI < 0.132910490561199


In [148]:
#Q24
issuccessratio = ad_recs_annotated[['region','rec_provider','is_success_binary']].groupby(['region','rec_provider'])['is_success_binary'].sum() / ad_recs_annotated[['region','rec_provider','is_success_binary','is_severe_corrected']].groupby(['region','rec_provider'])['is_success_binary'].size()
CImin = issuccessratio.mean() - ((1.96*issuccessratio.std())/len(issuccessratio))
CImax = issuccessratio.mean() + ((1.96*issuccessratio.std())/len(issuccessratio))
print(f'Range of confidence index for is_success: {CImin} < CI < {CImax}')

Range of confidence index for is_success: 0.8021431028283145 < CI < 0.8144659382915301


In [ ]:
# Q25
ad_recs_annotated[['requester','is_success_binary']].query("is_success_binary == 1").value_counts().head(1)

requester               is_success_binary
Search Million Culture  1                    70331
dtype: int64

In [ ]:
#Q29
#px.bar(ad_recs_annotated, x = 'week_id' ,y = 'requester', color='requester')

### Merges and joins

The Wow samples and annotations task were sent and recieve seperatly, per each week. The dataset you were working on was constructed from those files. You can see the files under the `data/weekly` folder, here are the first 10 files:

In [ ]:
import os
weekly_files = os.listdir('data/weekly/')
sorted(weekly_files)[:10]

FileNotFoundError: [WinError 3] ‏‏למערכת אין אפשרות לאתר את הנתיב שצוין: 'data/weekly/'

Your task is to reconstruct the dataset above. 

### Visualizations

Produce the following success rate graph per region:

: 

Produce the following requestser proportion graph:

: 

## Appendix

Code to create weekly files:

In [ ]:
# for l in ad_recs_annotated.region.unique():
#     for w in ad_recs_annotated.week_id.unique():
#         w_id = w.split('_')[1]
#         y = w.split('-')[0]
#         query = f'week_id == "{w}" and region == "{l}"'
#         sample_f_name = f'{y}_{w_id}_{l}__Sample.csv'
#         ad_recs_annotated.query(query)[['week_id','ad_id','requester','region','recommendation_type', 'rec_provider']].to_csv(f'data/weekly/{sample_f_name}', index=False)
#         for s in ad_recs_annotated.recommendation_type.unique():
#             ann_f_name = f'{y}_{w_id}_{l}_{s}_annotation_result.csv'
#             query = f'week_id == "{w}" and region == "{l}" and recommendation_type == "{s}"'
#             sss = ad_recs_annotated.query(query)
#             if sss.empty:
#                 continue            
#             sss[['region','ad_id','is_success','is_sever']].drop_duplicates(subset=['ad_id']).to_csv(f'data/weekly/{ann_f_name}', index=False)

: 